JuanD Valenciano.
jvalenciano@unal.edu.co

//TODO. documentar............................


In [25]:
import os
######
import sys
from sys import platform  #Detect platform!
#sys.path.append('/content/drive/My Drive/Tesis/DatosAnalizar/ColabLib')    #Dir Lib!
######

#from numba import jit
import time
import torch 
import numpy as np
import itertools

import spectral.io.envi as envi
from spectral import *
import matplotlib
import matplotlib.pyplot as plt
import wx
from wx import glcanvas
import warnings
import pandas as pd
#from IPython.html.widgets import *
from ipywidgets import *
#from IPython.html.widgets import interact # IPython < 4.x
#import envi2
#from envi2.constants import *

import math
import pywt
import pywt.data

from sklearn.metrics import mean_squared_error as MSE
from scipy.stats     import median_absolute_deviation as MAD
from skimage.restoration import (denoise_wavelet, estimate_sigma)
from skimage import data, img_as_float
from skimage.util import random_noise
from skimage import measure
##Error extrano al intentar cargar la libreria?/????
#from skimage.measure import compare_psnr
from skimage.exposure import rescale_intensity

import cv2


'''
import LogWrite
from LogWrite.logWriter import *
import cv2
from google.colab import output
import time
from tqdm import tqdm
'''

'''
import pysptools.util as util
import pysptools.noise as ns
import pysptools.skl as skl
'''
from numpy.linalg import matrix_power




class UnsupportedPlatform(Exception):
    pass